<a href="https://colab.research.google.com/github/imransalam/basic-tf-intro-notebooks/blob/master/tf2_0_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## some tensorflow2.0 stuff....

---

Please go to runtime type and change the environment to Python3

---

You can change the hardware accelerator to GPU too

---
The code written below does not follow "Good Coding Practices". This was made for a step by step guide. Hence a lot of things would be repeating.




In [0]:
!pip install tensorflow==2.0.0-alpha0
import tensorflow as tf
print(tf.__version__) # SHOULD BE 2.0.0-alpha0
print(tf.executing_eagerly()) # This should be true. Eager Execution skips the old graph-session part.

    100% |████████████████████████████████| 79.9MB 397kB/s 
    100% |████████████████████████████████| 3.0MB 10.4MB/s 
    100% |████████████████████████████████| 61kB 23.6MB/s 
    100% |████████████████████████████████| 419kB 20.0MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1
2.0.0-alpha0
True


## Tensors

In [0]:
'''
    Some basic Mathematics Operations Over.... something....
'''

# No need for graphs :/

x = tf.random.normal(shape=(3,3))
y = tf.random.normal(shape=(3,3))
z = tf.ones(shape=(1))

result = z + x * y
print(result)

# Wait, that's like numpy?!?!?!
# You guessed it ;)

tf.Tensor(
[[ 0.7757539   0.9024863   1.8981946 ]
 [-0.28960025  1.3480455   1.1079812 ]
 [-0.46879113  1.0354439   2.1613226 ]], shape=(3, 3), dtype=float32)


## Variables

In [0]:
'''
    Some basic Mathematics Operations Over Placeholders and Variables
'''

# But you lied to us, you said this was just like numpy.
# Yes, I lied, But we need to train something, like optimize a variable, CHANGE ITS STATE FROM RANDOM TO SOMETHING USEFUL

x = tf.random.normal(shape=(500, 8)) # 500 Examples, 8 Features.

w = tf.Variable(tf.random.normal(shape=(8, 3))) # (500x8)*(8x3)=(500,3), We assume that we have 3 classes 
b = tf.Variable(tf.random.normal(shape=(3,)))

y_pred = tf.matmul(x , w) + b

print(y_pred)

tf.Tensor(
[[ 1.6798445   2.079223    5.089149  ]
 [-1.3021377   3.3971395   0.1161238 ]
 [ 2.8522253  -0.15051036  2.1029546 ]
 ...
 [-3.3952367  -1.1727492  -3.6189575 ]
 [ 1.0804975  -1.261142    0.9648686 ]
 [-1.5507914   3.0282185   1.7486347 ]], shape=(500, 3), dtype=float32)


## KICK SOME TRAINING!!!!!!!!!!!!!!!!!!!

In [0]:
'''
    Training Over Boston Housing. Its a simple Linear regression :/
'''

# LOAD DATASET
dataset = tf.keras.datasets.boston_housing
(x_train, y_train), (x_val, y_val) = dataset.load_data()

# Let's Build a model now!

class LinearRegression():
    def __init__(self): # Define All your Variables Here. And other configurations
        self.lr = 1e-7
        self.show_every = 1000
        self.epochs = 10000
        self.W = tf.Variable(tf.random.normal(shape=(13, 1), dtype=tf.float64), name='W') # (500x8)*(8x3)=(500,3), We assume that we have 3 classes 
        self.b = tf.Variable(tf.random.normal(shape=(1,), dtype=tf.float64), name='b')
    
    def predict(self, x): # Use the Configurations and the variables defined here.... this is forward prop
        return tf.matmul(x, self.W) + self.b
    
    def loss(self, predicted_y, desired_y): # The loss function we are going to be using
        return tf.reduce_mean(tf.square(predicted_y - desired_y))
        
    
    def fit(self, x, y, val_x, val_y): # Training Loop        
        def train_pipe():
            with tf.GradientTape() as tape:
                loss = self.loss(self.predict(x), y)
            dW, db = tape.gradient(loss, [self.W, self.b]) #### WAIT
            self.W.assign_sub(self.lr * dW) #### UMM.... WHAT?!?!
            self.b.assign_sub(self.lr * db) #### This is gradient Descent... don't worry.
            return loss

        for epoch in range(self.epochs):
            cur_loss = train_pipe()
            if epoch % 500 == 0:
                val_loss = self.loss(self.predict(val_x), val_y)
                print('Epoch Number: ', epoch, ' Loss is: ', cur_loss.numpy() / x.shape[0])
                print('Epoch Number: ', epoch, ' Validation Loss is: ', val_loss.numpy() / val_x.shape[0])
                print('-'*20)
            
            
            
    

model = LinearRegression()
model.fit(x_train, y_train, x_val, y_val)

57344/57026 [==============================] - 0s 0us/step
Epoch Number:  0  Loss is:  1020.8039590949929
Epoch Number:  0  Validation Loss is:  3738.635093277587
--------------------
Epoch Number:  500  Loss is:  2.8412234581556994
Epoch Number:  500  Validation Loss is:  8.820767623170394
--------------------
Epoch Number:  1000  Loss is:  2.240594552556054
Epoch Number:  1000  Validation Loss is:  6.9315357930725465
--------------------
Epoch Number:  1500  Loss is:  1.97180183587482
Epoch Number:  1500  Validation Loss is:  6.065736987648328
--------------------
Epoch Number:  2000  Loss is:  1.7449948304748388
Epoch Number:  2000  Validation Loss is:  5.332053684677029
--------------------
Epoch Number:  2500  Loss is:  1.5518609400363683
Epoch Number:  2500  Validation Loss is:  4.710765624502969
--------------------
Epoch Number:  3000  Loss is:  1.3872659475178022
Epoch Number:  3000  Validation Loss is:  4.184923049929645
--------------------
Epoch Number:  3500  Loss is:  1.2

### THIS IS MORE DIFFICULT THAN TF1.0 ?!?!?!
### I know I made it so ;)
## Let's see the real magic of TF2.0 with KERAS

In [0]:
'''
    Simple Linear Regression Code with the power of TF with Keras
    Training Over Boston Housing. Its a simple Linear regression :/
'''

# LOAD DATASET
dataset = tf.keras.datasets.boston_housing
(x_train, y_train), (x_val, y_val) = dataset.load_data()

# Let's Build a model now!

class LinearRegression(tf.keras.Model): # Subclass from tf.keras.model
    def __init__(self): # Define All your Variables Here. And other configurations
        super(LinearRegression, self).__init__()
        self.dense = tf.keras.layers.Dense(1)
    
    def call(self, x): # Use the variables defined here.... this is forward prop
        return self.dense(x)            
    

model = LinearRegression()
model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train, epochs=10000, verbose=0)
model.evaluate(x_val, y_val) / x_val.shape[0]

102/102 [==============================] - 0s 55us/sample - loss: 23.1735


0.22719119026861664

### BAM!!!!!!!

## Define Custom Loss

In [0]:
'''
    We are going to do Linear Regression with Huber Loss
'''
# LOAD DATASET
dataset = tf.keras.datasets.boston_housing
(x_train, y_train), (x_val, y_val) = dataset.load_data()

# Let's Build a model now!

def huber_loss(y_true, y_pred, delta):
    tf_delta = tf.constant(value=delta, dtype=tf.float32)
    residual = tf.reduce_mean(tf.abs(y_pred - y_true))
    condition = tf.less(residual, tf_delta)
    small_res = 0.5 * tf.square(residual)
    large_res = tf_delta * residual - 0.5 * tf.square(tf_delta)
    return tf.where(condition, small_res, large_res)

def get_loss(delta=1.0):
    def huber(y_true, y_pred):
        return huber_loss(y_true, y_pred, delta)
    return huber

class LinearRegression(tf.keras.Model): # Subclass from tf.keras.model
    def __init__(self): # Define All your Variables Here. And other configurations
        super(LinearRegression, self).__init__()
        self.dense = tf.keras.layers.Dense(1)
    
    def call(self, x): # Use the variables defined here.... this is forward prop
        return self.dense(x)            
    
my_custom_loss = get_loss()
model = LinearRegression()
model.compile(loss=my_custom_loss, optimizer='adam')
model.fit(x_train, y_train, epochs=10000, verbose=0 )
model.evaluate(x_val, y_val) / x_val.shape[0]

102/102 [==============================] - 0s 174us/sample - loss: 2.6805


0.026279310041645406

## Classification!

In [0]:
'''
    Training Over MNIST
'''

# LOAD DATASET

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

class LogisticRegression(tf.keras.Model): # Subclass from tf.keras.model
    def __init__(self): # Define All your Variables Here. And other configurations
        super(LogisticRegression, self).__init__()
        self.dense = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        self.flatten_layer = tf.keras.layers.Flatten(input_shape=(28, 28))
    
    def call(self, x): # Use the variables defined here.... this is forward prop
        x = self.flatten_layer(x)
        x = self.dense(x)
        return x
        

model = LogisticRegression()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)


Epoch 1/5
60000/60000 [==============================] - 3s 49us/sample - loss: 1.6382 - accuracy: 0.5839
Epoch 2/5
60000/60000 [==============================] - 3s 46us/sample - loss: 1.0353 - accuracy: 0.7996
Epoch 3/5
60000/60000 [==============================] - 3s 46us/sample - loss: 0.8233 - accuracy: 0.8286
Epoch 4/5
60000/60000 [==============================] - 3s 46us/sample - loss: 0.7159 - accuracy: 0.8422
Epoch 5/5
10000/10000 [==============================] - 0s 36us/sample - loss: 0.5995 - accuracy: 0.8636


[0.5994680000305176, 0.8636]

## One Hidden Layer

In [0]:
'''
    Training Over MNIST
'''

# LOAD DATASET

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

class OneLayeredNNClassification(tf.keras.Model): # Subclass from tf.keras.model
    def __init__(self): # Define All your Variables Here. And other configurations
        super(OneLayeredNNClassification, self).__init__()
        self.dense_final = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        self.dense = tf.keras.layers.Dense(512, activation=tf.nn.relu)
        self.flatten_layer = tf.keras.layers.Flatten(input_shape=(28, 28))
    
    def call(self, x): # Use the variables defined here.... this is forward prop
        x = self.flatten_layer(x)
        x = self.dense(x)
        x = self.dense_final(x)
        return x
        

model = OneLayeredNNClassification()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)


Epoch 1/5
60000/60000 [==============================] - 8s 133us/sample - loss: 1.5514 - accuracy: 0.6519
Epoch 2/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.8346 - accuracy: 0.8310
Epoch 3/5
60000/60000 [==============================] - 8s 137us/sample - loss: 0.6205 - accuracy: 0.8582
Epoch 4/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.5249 - accuracy: 0.8716
Epoch 5/5
10000/10000 [==============================] - 1s 78us/sample - loss: 0.4303 - accuracy: 0.8901


[0.4302520763158798, 0.8901]

## Stack More Layers

In [0]:
'''
    Training Over MNIST
'''

# LOAD DATASET

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

class NNClassification(tf.keras.Model): # Subclass from tf.keras.model
    def __init__(self): # Define All your Variables Here. And other configurations
        super(NNClassification, self).__init__()
        self.dense_final = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.flatten_layer = tf.keras.layers.Flatten(input_shape=(28, 28))
    
    def call(self, x): # Use the variables defined here.... this is forward prop
        x = self.flatten_layer(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense_final(x)
        return x
        

model = NNClassification()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)


Epoch 1/5
60000/60000 [==============================] - 8s 125us/sample - loss: 1.9297 - accuracy: 0.4965
Epoch 2/5
60000/60000 [==============================] - 7s 116us/sample - loss: 0.9364 - accuracy: 0.7970
Epoch 3/5
60000/60000 [==============================] - 7s 119us/sample - loss: 0.5654 - accuracy: 0.8556
Epoch 4/5
60000/60000 [==============================] - 7s 124us/sample - loss: 0.4462 - accuracy: 0.8808
Epoch 5/5
10000/10000 [==============================] - 1s 76us/sample - loss: 0.3532 - accuracy: 0.9000


[0.3532075037121773, 0.9]

## ConvNets - Saving a Model

In [0]:
'''
    Training Over MNIST
'''

# LOAD DATASET

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

class ConvNetClassification(tf.keras.Model): # Subclass from tf.keras.model
    def __init__(self): # Define All your Variables Here. And other configurations
        super(ConvNetClassification, self).__init__()
        self.conv2d_1 = tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu')
        self.max2d_1 = tf.keras.layers.MaxPool2D((3,3), padding='valid', strides=(2,2))
        self.conv2d_2 = tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='relu')
        self.max2d_2 = tf.keras.layers.MaxPool2D((3,3), padding='valid', strides=(2,2))
        
        self.flatten_layer = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1000, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense_final = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    
    def call(self, x): # Use the variables defined here.... this is forward prop
        x = self.conv2d_1(x)
        x = self.max2d_1(x)
        x = self.conv2d_2(x)
        x = self.max2d_2(x)
        x = self.flatten_layer(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense_final(x)
        return x
        

model = ConvNetClassification()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1, batch_size=4)
model.evaluate(x_test, y_test)


model.save_weights('My-ConvNet-Model.h5')


10000/10000 [==============================] - 8s 819us/sample - loss: 0.0580 - accuracy: 0.9828


NotImplementedError: ignored

## Load Model

In [0]:
class ConvNetClassification(tf.keras.Model): # Subclass from tf.keras.model
    def __init__(self): # Define All your Variables Here. And other configurations
        super(ConvNetClassification, self).__init__()
        self.conv2d_1 = tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu')
        self.max2d_1 = tf.keras.layers.MaxPool2D((3,3), padding='valid', strides=(2,2))
        self.conv2d_2 = tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='relu')
        self.max2d_2 = tf.keras.layers.MaxPool2D((3,3), padding='valid', strides=(2,2))
        
        self.flatten_layer = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1000, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense_final = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    
    def call(self, x): # Use the variables defined here.... this is forward prop
        x = self.conv2d_1(x)
        x = self.max2d_1(x)
        x = self.conv2d_2(x)
        x = self.max2d_2(x)
        x = self.flatten_layer(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense_final(x)
        return x
    
model = ConvNetClassification()
model.load_weights('My-ConvNet-Model.h5')